In [1]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import re
import uuid 
import json
import sqlite3

pd.set_option('display.max_rows', 900)

In [2]:
class BETS():
    def __init__(self):
        load_dotenv()
        self.clientId = os.getenv('CLIENT_ID')
        self.clientPw = os.getenv('CLIENT_PW')
        self.baseUrl = "https://api.tournamentsoftware.com/1.0"
    
    def make_call(self,url):
        response = requests.get(url, auth=HTTPBasicAuth(self.clientId, self.clientPw))
        return response

    #   USE SEARCH QUERY '?q=U11 Gold'
    #   Filter by dates
    #   Create lists of cool tourney codes.
    def search_tournaments(self,dateStr):
        splitDate = dateStr.split('/')
        refDate = splitDate[2] + '-' + splitDate[1] + '-' + splitDate[0]
        url = self.baseUrl+f"/Tournament?list=1&refdate={refDate}&pagesize=1000" 
        return self.make_call(url)
    
    def get_tournament_details(self,tCode):
        url = self.baseUrl+f"/Tournament/{tCode}"
        return self.make_call(url)

    def get_matches(self,tCode,date):
        dateQuery = date.strftime("%Y%m%d")
        url = self.baseUrl+f"/Tournament/{tCode}/Match/{dateQuery}"
        return self.make_call(url)
    
    def wanted_rounds(self,roundCaredAbout):
        allRounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'Final']
        if roundCaredAbout == 'Winner':
            return ['Final']
        elif roundCaredAbout in allRounds:
            index = allRounds.index(roundCaredAbout)
            return allRounds[index:]
        else:
            return ['','','','','','','']


In [3]:
def processXMLList(list):
    out = []
    for xml in list:
        out.append(xml.text)
    return out

def get_exclude_list():
    tabuTitles = ['invite','(1000ie)','masters','para','yonex','league','regional','schools','graded','rising','handicap']
    return tabuTitles

def get_tourney_deets(title):
    title = title.lower()
    age = ''
    level = ''

    if 'tier 4' in title:
        level = 'Tier 4'
    if 'bronze' in title:
        level = 'Bronze'
    if 'silver' in title:
        level = 'Silver'
    if 'gold' in title:
        level = 'Gold'
    if 'national' in title:
        level = 'Nationals'
    if 'restricted' in title:
        level = 'Restricted'
    
    ageGroupRegex = re.compile('u(1[0-9])')
    ageMatch = re.search(ageGroupRegex,title)
    if ageMatch:
        age = ageMatch.group().upper()
    elif 'senior' in title:
        age = 'Senior'

    return age,level



#Both inputs as strings, searchStart = d/m/y, searchText = 'U11 Gold'
def get_tournament_results(bets,searchStartStr):
    tabuList = get_exclude_list()
    searchStart = datetime.strptime(searchStartStr,"%d/%m/%Y")
    tourneysDf = pd.DataFrame(columns=['Name','Code','StartDate','EndDate'])
    tourneysToAdd = []

    out = bets.search_tournaments(searchStartStr).content
    soup = BeautifulSoup(out,'xml')
    tournaments = soup.find_all('Tournament')
    for i in range(0,len(tournaments)):  
        name = tournaments[i].find('Name').text
        code = tournaments[i].find('Code').text
        startDate = datetime.fromisoformat(tournaments[i].find('StartDate').text)
        endDate = datetime.fromisoformat(tournaments[i].find('EndDate').text)
        tType = tournaments[i].find('TypeID').text
        
        goodTourney = True
        if not tType == '0':
            goodTourney = False
        
        for t in tabuList:
            if t in name.lower():
                goodTourney = False
        if not any(existing_tourney['Name'] == name for existing_tourney in tourneysToAdd) and goodTourney:
            tourney = {
                "Name": name,
                "Code": code,
                "StartDate": startDate,
                "EndDate": endDate,
            }

            tourneysToAdd.append(tourney)

    tourneysDf = pd.concat([tourneysDf, pd.DataFrame(tourneysToAdd)], ignore_index=True)
    searchUntil = datetime.today() - timedelta(days = 2)
    searchFrom = searchStart

    
    # Apply condition and filter rows
    tourneysInRangeDf = tourneysDf[(tourneysDf['StartDate'] <= searchUntil)].copy()

    linksDf = tourneysInRangeDf[['Name','Code','StartDate']]

    def code_to_link(code):
        return f'https://be.tournamentsoftware.com/tournament/{code}'
    
    linksDf['Code'] = linksDf['Code'].apply(code_to_link)


    #Itterate through Tournaments
    match_data = []
    for index, row in tourneysInRangeDf.iterrows():
        dtCurrentDay = row['StartDate']
        dtEndDate = row['EndDate']
        code = row['Code']
        tName = row['Name']
        

        
        #Itterate through Days With Matches.
        allMatchesXML = [] 
        while dtCurrentDay <= dtEndDate:
            tDetails = bets.get_matches(row['Code'],dtCurrentDay)
            matchesXML = tDetails.content
            allMatchesXML.append(matchesXML)
            dtCurrentDay = dtCurrentDay + timedelta(days=1)

        tAge,tLevel = get_tourney_deets(tName)
        # Iterate over each Match element
        for matchesXML in allMatchesXML:
            soup = BeautifulSoup(matchesXML,'xml')
            for match in soup.find_all('Match'):

                #check if match went ahead:
                if not match.find('ScoreStatus').text == '0':
                    pass
                else:
                    try:
                        roundName = match.find('RoundName').text
                    except:
                        roundName = 'Not Found'
                        print('no round name for match:',match)
                    match_info = {
                        'Tournament': tName,
                        'Date': dtCurrentDay,
                        'Age': tAge,
                        'Level':tLevel,
                        'Code': match.find('Code').text,
                        'Winner': match.find('Winner').text,
                        'ScoreStatus': match.find('ScoreStatus').text,
                        'RoundName': roundName,
                        'EventName': match.find('EventName').text,
                    }
                    # Extract team and player details
                    teams = []
                    for team in ['Team1', 'Team2']:
                        teamXML = match.find(team)
                        team_info = {}
                        #find player id's and if singles
                        team_info[f'{team}_MemberID'] = processXMLList(teamXML.find_all('MemberID'))
                        team_info[f'{team}_Firstname'] = processXMLList(teamXML.find_all('Firstname'))
                        team_info[f'{team}_Lastname'] = processXMLList(teamXML.find_all('Lastname'))
                        team_info[f'{team}_GenderID'] = processXMLList(teamXML.find_all('GenderID'))

                        teams.append(team_info)

                    match_info.update(teams[0])
                    match_info.update(teams[1])

                    # Extract sets scores
                    sets = match.find('Sets')
                    set_scores = []
                    if sets:
                        for set_ in sets.find_all('Set'):
                            set_scores.append({
                                'Set_Team1': set_['Team1'],
                                'Set_Team2': set_['Team2']
                            })
    
                        match_info['Set_Scores'] = set_scores
                    

                    # Append the structured match information to the list
                    match_data.append(match_info)

            # Convert the list to a DataFrame
    df = pd.DataFrame(match_data)
    return df,linksDf


In [4]:
bets = BETS()
matchData,linksDf = get_tournament_results(bets,'15/01/2025') #d m y

/tmp/ipykernel_8215/251674033.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linksDf['Code'] = linksDf['Code'].apply(code_to_link)


In [5]:
matchData.head(1)

,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,Team1_Firstname,Team1_Lastname,Team1_GenderID,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores
0,SBA U17 Silver Jan,2025-01-26,U17,Silver,232,1,0,R1,XD U17,"[1359691, 1346679]","[Rory, Ava]","[Le Masurier, Dodkins]","[1, 2]","[1373984, 1383714]","[Ethan, Nidhii]","[Jamison, Deepak]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."


In [6]:
# Format names
for team in ['Team1','Team2']:
    namesFormatted = []
    for index, row in matchData.iterrows():
        firstNames = row[f'{team}_Firstname']
        secondNames = row[f'{team}_Lastname']

        if not (len(firstNames) == len(secondNames)):
            raise ValueError(f'There are not matching numbers of first and second names when formatting.\n See below for row \n {row}')

        playerNames = []
        if len (firstNames) > 1: #Doubles
            
            for i in range(0,len(firstNames)):
                playerNames.append(firstNames[i] + ' ' +  secondNames[i])

        else: #Singles
            playerNames.append(firstNames[0] + ' ' +  secondNames[0])

        namesFormatted.append(playerNames)

    matchData[f'{team}_Names'] = namesFormatted


## Drop the original columns
#df = df.drop(['Col1', 'Col2'], axis=1)
#
#print(df)



In [7]:
#Format Event
matchData['Event'] = matchData['EventName'].str[:2]

#Format Gender
def replace_gender(value):
    if isinstance(value, list):  # Check if the entry is a list
        return ['M' if v == '1' else 'F' for v in value]
    return 'M' if value == '1' else 'F'

def replace_round(value):
    if 'R' in value:
        roundNum = value[1:]
        if roundNum in ['16','32','128','256']:
            return 'Round Of ' + roundNum
        return 'Round ' + value[1:]
    if value == 'SF':
        return 'Semi-Final'
    if value == 'QF':
        return 'Quarter-Final'
    if value == 'Final':
        return value

# Apply the replacement
for team in ['Team1','Team2']:
    matchData[f'{team}_Gender'] = matchData[f'{team}_GenderID'].apply(replace_gender)

# Create list of all ID's for quick indexing
allIds = []
for index,row in matchData.iterrows():
    rowIds = []
    for team in ['Team1','Team2']:
        for id in row[f'{team}_MemberID']:
            rowIds.append(id)
    allIds.append(rowIds)
matchData['PID'] = allIds

matchData['Round'] = matchData['RoundName'].apply(replace_round)

matchData#.head(5)


,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,...,Team2_Lastname,Team2_GenderID,Set_Scores,Team1_Names,Team2_Names,Event,Team1_Gender,Team2_Gender,PID,Round
0,SBA U17 Silver Jan,2025-01-26,U17,Silver,232,1,0,R1,XD U17,"[1359691, 1346679]",...,"[Jamison, Deepak]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_...","[Rory Le Masurier, Ava Dodkins]","[Ethan Jamison, Nidhii Deepak]",XD,"[M, F]","[M, F]","[1359691, 1346679, 1373984, 1383714]",Round 1
1,SBA U17 Silver Jan,2025-01-26,U17,Silver,488,2,0,R1,OS U17,[1370417],...,[Chan],[1],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set...",[Vishnu Bandari],[Jasper Chan],OS,[M],[M],"[1370417, 1381811]",Round 1
2,SBA U17 Silver Jan,2025-01-26,U17,Silver,486,2,0,R1,OS U17,[1360259],...,[Shah],[1],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set...",[Shiv Vitish],[Ahmed Shah],OS,[M],[M],"[1360259, 1374085]",Round 1
3,SBA U17 Silver Jan,2025-01-26,U17,Silver,223,2,0,R1,XD U17,"[1373449, 1370884]",...,"[Kwiecien, Gautam]","[1, 2]","[{'Set_Team1': '16', 'Set_Team2': '18'}, {'Set...","[Leo Zhu, Diya Karthik]","[Olaf Kwiecien, Avyukta Gautam]",XD,"[M, F]","[M, F]","[1373449, 1370884, 1360084, 1380214]",Round 1
4,SBA U17 Silver Jan,2025-01-26,U17,Silver,504,1,0,R1,OS U17,[1370797],...,[Jamison],[1],"[{'Set_Team1': '15', 'Set_Team2': '11'}, {'Set...",[Mitulsaran Udayan],[Ethan Jamison],OS,[M],[M],"[1370797, 1373984]",Round 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,338,1,0,QF,OD U17,"[1386211, 1371654]",...,"[Chan, Chu]","[1, 1]","[{'Set_Team1': '21', 'Set_Team2': '15'}]","[Hayden Lau, Adam Lesler]","[Yik Wang Ethan Chan, Andre Chu]",OD,"[M, M]","[M, M]","[1386211, 1371654, 1382566, 1377504]",Quarter-Final
1460,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,339,2,0,QF,OD U17,"[1372067, 1351154]",...,"[Chee Keong, Shah]","[1, 1]","[{'Set_Team1': '12', 'Set_Team2': '21'}]","[William Kimber, Finlay Wee]","[Yeoh Chee Keong, Ahmed Shah]",OD,"[M, M]","[M, M]","[1372067, 1351154, 1348438, 1374085]",Quarter-Final
1461,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,335,1,0,SF,OD U17,"[1351728, 1346056]",...,"[Lau, Lesler]","[1, 1]","[{'Set_Team1': '21', 'Set_Team2': '18'}]","[Etienne Fan, Eden Gough]","[Hayden Lau, Adam Lesler]",OD,"[M, M]","[M, M]","[1351728, 1346056, 1386211, 1371654]",Semi-Final
1462,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,336,1,0,SF,OD U17,"[1348438, 1374085]",...,"[Cundall, Randall-Coles]","[1, 1]","[{'Set_Team1': '22', 'Set_Team2': '20'}]","[Yeoh Chee Keong, Ahmed Shah]","[Max Cundall, Tom Randall-Coles]",OD,"[M, M]","[M, M]","[1348438, 1374085, 1339142, 1365296]",Semi-Final


In [8]:
explosionTest = matchData.explode('PID')

In [9]:
#For later: 'Tournament','Date','Age','Level', 'RoundName'
wantedCols = ['PID','Tournament','Date','Age','Level','Event','Round','Winner','Team1_MemberID','Team1_Names','Team1_Gender','Team2_MemberID','Team2_Names','Team2_Gender','Set_Scores']

explosionTest = explosionTest[wantedCols]
explosionTest

,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,1359691,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
0,1346679,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
0,1373984,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
0,1383714,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
1,1370417,SBA U17 Silver Jan,2025-01-26,U17,Silver,OS,Round 1,2,[1370417],[Vishnu Bandari],[M],[1381811],[Jasper Chan],[M],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,1365296,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,OD,Semi-Final,1,"[1348438, 1374085]","[Yeoh Chee Keong, Ahmed Shah]","[M, M]","[1339142, 1365296]","[Max Cundall, Tom Randall-Coles]","[M, M]","[{'Set_Team1': '22', 'Set_Team2': '20'}]"
1463,1351728,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,OD,Final,1,"[1351728, 1346056]","[Etienne Fan, Eden Gough]","[M, M]","[1348438, 1374085]","[Yeoh Chee Keong, Ahmed Shah]","[M, M]","[{'Set_Team1': '21', 'Set_Team2': '17'}]"
1463,1346056,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,OD,Final,1,"[1351728, 1346056]","[Etienne Fan, Eden Gough]","[M, M]","[1348438, 1374085]","[Yeoh Chee Keong, Ahmed Shah]","[M, M]","[{'Set_Team1': '21', 'Set_Team2': '17'}]"
1463,1348438,Swindon Stars U17 Bronze Jan 2025,2025-01-19,U17,Bronze,OD,Final,1,"[1351728, 1346056]","[Etienne Fan, Eden Gough]","[M, M]","[1348438, 1374085]","[Yeoh Chee Keong, Ahmed Shah]","[M, M]","[{'Set_Team1': '21', 'Set_Team2': '17'}]"


In [10]:
def make_player_in_team1(row):
    if str(row['PID']) in row['Team2_MemberID']:  # If the player is in Team2, swap
        # Swap all relevant columns
        row['Team1_MemberID'], row['Team2_MemberID'] = row['Team2_MemberID'], row['Team1_MemberID']
        row['Team1_Names'], row['Team2_Names'] = row['Team2_Names'], row['Team1_Names']
        row['Team1_Gender'], row['Team2_Gender'] = row['Team2_Gender'], row['Team1_Gender']
        
        # Swap set scores (Team1 ↔ Team2)
        if isinstance(row['Set_Scores'], list):
            swapped_scores = []
            for score in row['Set_Scores']:
                # Ensure 'Set_Team1' and 'Set_Team2' keys exist before swapping
                swapped_scores.append({'Set_Team1': score['Set_Team2'], 'Set_Team2': score['Set_Team1']})
            row['Set_Scores'] = swapped_scores
        
        # Update winner: If originally Team2 won (Winner = 2), set Winner = 1, and vice versa
        prevWinner = int(row['Winner'])
        row['Winner'] = 1 if prevWinner == 2 else 2

    return row

# Apply the function to each row
preCorrectedDf = explosionTest.copy()
correctedDf = explosionTest.apply(make_player_in_team1, axis=1)


In [11]:
correctedDf.head(8)

,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,1359691,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
0,1346679,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,1,"[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."
0,1373984,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,2,"[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[{'Set_Team1': '7', 'Set_Team2': '15'}, {'Set_..."
0,1383714,SBA U17 Silver Jan,2025-01-26,U17,Silver,XD,Round 1,2,"[1373984, 1383714]","[Ethan Jamison, Nidhii Deepak]","[M, F]","[1359691, 1346679]","[Rory Le Masurier, Ava Dodkins]","[M, F]","[{'Set_Team1': '7', 'Set_Team2': '15'}, {'Set_..."
1,1370417,SBA U17 Silver Jan,2025-01-26,U17,Silver,OS,Round 1,2,[1370417],[Vishnu Bandari],[M],[1381811],[Jasper Chan],[M],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set..."
1,1381811,SBA U17 Silver Jan,2025-01-26,U17,Silver,OS,Round 1,1,[1381811],[Jasper Chan],[M],[1370417],[Vishnu Bandari],[M],"[{'Set_Team1': '15', 'Set_Team2': '13'}, {'Set..."
2,1360259,SBA U17 Silver Jan,2025-01-26,U17,Silver,OS,Round 1,2,[1360259],[Shiv Vitish],[M],[1374085],[Ahmed Shah],[M],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set..."
2,1374085,SBA U17 Silver Jan,2025-01-26,U17,Silver,OS,Round 1,1,[1374085],[Ahmed Shah],[M],[1360259],[Shiv Vitish],[M],"[{'Set_Team1': '15', 'Set_Team2': '13'}, {'Set..."


### The following is for database dramas

In [18]:
# Copy DataFrame to avoid modifying the original
df = explosionTest.copy()

# Function to generate unique MatchID
def generate_match_id(row):
    date = row['Date']
    
    # Get first player ID from Team1_MemberID (assuming it's a list)
    first_team1_id = str(row['Team1_MemberID']) if isinstance(row['Team1_MemberID'], list) else str(row['Team1_MemberID'])
    
    # Convert Team2_MemberID to a string (flatten if it's a list)
    team2_id = ''.join(map(str, row['Team2_MemberID'])) if isinstance(row['Team2_MemberID'], list) else str(row['Team2_MemberID'])
    
    event = row['Event']
    
    # Concatenate all parts into a unique string
    return f"{date}_{first_team1_id}_{team2_id}_{event}"

# Apply function to create MatchID column
df.insert(0, 'MatchID', df.apply(generate_match_id, axis=1))

for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (list, dict))).any():
        df[col] = df[col].apply(json.dumps) 

# Database file name
db_name = "local_database.db"

# Function to save DataFrame to SQLite database
def save_to_database(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='append', index=False)
    print(f"DataFrame saved to table '{table_name}' in database '{db_name}'.")
    conn.close()

# Save DataFrame to the database
save_to_database(df, db_name, table_name="your_table")



DataFrame saved to table 'your_table' in database 'local_database.db'.


In [19]:
# Load the DataFrame from SQLite
conn = sqlite3.connect(db_name)
df_loaded = pd.read_sql(f"SELECT * FROM your_table", conn)

## Convert JSON strings back to lists/dicts
#for col in df_loaded.columns:
#    if df_loaded[col].apply(lambda x: isinstance(x, str) and x.startswith("[") or x.startswith("{") ).any():
#        df_loaded[col] = df_loaded[col].apply(json.loads)


conn.close()

df_loaded

,MatchID,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,"2025-01-26 00:00:00_['1359691', '1346679']_137...",1359691,SBA U17 Silver Jan,2025-01-26 00:00:00,U17,Silver,XD,Round 1,1,"[""1359691"", ""1346679""]","[""Rory Le Masurier"", ""Ava Dodkins""]","[""M"", ""F""]","[""1373984"", ""1383714""]","[""Ethan Jamison"", ""Nidhii Deepak""]","[""M"", ""F""]","[{""Set_Team1"": ""15"", ""Set_Team2"": ""7""}, {""Set_..."
1,"2025-01-26 00:00:00_['1359691', '1346679']_137...",1346679,SBA U17 Silver Jan,2025-01-26 00:00:00,U17,Silver,XD,Round 1,1,"[""1359691"", ""1346679""]","[""Rory Le Masurier"", ""Ava Dodkins""]","[""M"", ""F""]","[""1373984"", ""1383714""]","[""Ethan Jamison"", ""Nidhii Deepak""]","[""M"", ""F""]","[{""Set_Team1"": ""15"", ""Set_Team2"": ""7""}, {""Set_..."
2,"2025-01-26 00:00:00_['1359691', '1346679']_137...",1373984,SBA U17 Silver Jan,2025-01-26 00:00:00,U17,Silver,XD,Round 1,1,"[""1359691"", ""1346679""]","[""Rory Le Masurier"", ""Ava Dodkins""]","[""M"", ""F""]","[""1373984"", ""1383714""]","[""Ethan Jamison"", ""Nidhii Deepak""]","[""M"", ""F""]","[{""Set_Team1"": ""15"", ""Set_Team2"": ""7""}, {""Set_..."
3,"2025-01-26 00:00:00_['1359691', '1346679']_137...",1383714,SBA U17 Silver Jan,2025-01-26 00:00:00,U17,Silver,XD,Round 1,1,"[""1359691"", ""1346679""]","[""Rory Le Masurier"", ""Ava Dodkins""]","[""M"", ""F""]","[""1373984"", ""1383714""]","[""Ethan Jamison"", ""Nidhii Deepak""]","[""M"", ""F""]","[{""Set_Team1"": ""15"", ""Set_Team2"": ""7""}, {""Set_..."
4,2025-01-26 00:00:00_['1370417']_1381811_OS,1370417,SBA U17 Silver Jan,2025-01-26 00:00:00,U17,Silver,OS,Round 1,2,"[""1370417""]","[""Vishnu Bandari""]","[""M""]","[""1381811""]","[""Jasper Chan""]","[""M""]","[{""Set_Team1"": ""13"", ""Set_Team2"": ""15""}, {""Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,"2025-01-19 00:00:00_['1348438', '1374085']_133...",1365296,Swindon Stars U17 Bronze Jan 2025,2025-01-19 00:00:00,U17,Bronze,OD,Semi-Final,1,"[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[""1339142"", ""1365296""]","[""Max Cundall"", ""Tom Randall-Coles""]","[""M"", ""M""]","[{""Set_Team1"": ""22"", ""Set_Team2"": ""20""}]"
4059,"2025-01-19 00:00:00_['1351728', '1346056']_134...",1351728,Swindon Stars U17 Bronze Jan 2025,2025-01-19 00:00:00,U17,Bronze,OD,Final,1,"[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}]"
4060,"2025-01-19 00:00:00_['1351728', '1346056']_134...",1346056,Swindon Stars U17 Bronze Jan 2025,2025-01-19 00:00:00,U17,Bronze,OD,Final,1,"[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}]"
4061,"2025-01-19 00:00:00_['1351728', '1346056']_134...",1348438,Swindon Stars U17 Bronze Jan 2025,2025-01-19 00:00:00,U17,Bronze,OD,Final,1,"[""1351728"", ""1346056""]","[""Etienne Fan"", ""Eden Gough""]","[""M"", ""M""]","[""1348438"", ""1374085""]","[""Yeoh Chee Keong"", ""Ahmed Shah""]","[""M"", ""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}]"
